# Employing Ridge Regression
*Curtis Miller*

As mentioned before, OLS has a propensity to find spurious fits. **Ridge regression** helps prevent this by introducing regularization parameters, such as $\alpha$. Larger $\alpha$ implies more regularization and less overfitting.

Ridge regression is implemented via the `Ridge` object provided in **scikit-learn**.

We will work with the Boston housing price dataset. In an earlier video we saw that removing some features may lead to better models. We will remove those features here as well.

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
boston_obj = load_boston()
data_train, data_test, price_train, price_test = train_test_split(boston_obj.data, boston_obj.target)
data_train = np.delete(data_train, [2, 6], axis=1)
data_test = np.delete(data_test, [2, 6], axis=1)

data_train[:5, :]

## Fitting With Ridge Regression

Ridge regression is implemented in the `Ridge` object supplied by **scikit-learn**.

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [ ]:
ridge1 = Ridge(alpha=1)    # alpha is a hyperparameter controlling regularization
ridge1.fit(data_train, price_train)
ridge1.predict([[    # An example prediction
    1,      # Per capita crime rate
    25,     # Proportion of land zoned for large homes
    1,      # Tract bounds the Charles River
    0.3,    # NOX concentration
    10,     # Average number of rooms per dwelling
    10,     # Weighted distance to employment centers
    3,      # Index for highway accessibility
    400,    # Tax rate
    15,     # Pupil/teacher ratio
    200,    # Index for number of blacks
    5       # % lower status of population
]])

In [ ]:
predprice = ridge1.predict(data_train)
mean_squared_error(price_train, predprice)

We can use cross-validation to pick a good value for $\alpha$. I will use `cross_val_score()` for this work.

In [ ]:
import pandas as pd
from pandas import DataFrame

In [ ]:
alpha = [.125, .25, .5, 1, 2, 4, 8, 16, 32, 64, 128]    # Candidate alphas
res = dict()

for a in alpha:
    ridge2 = Ridge(alpha=a)
    res[a] = cross_val_score(ridge2, data_train, price_train, scoring='neg_mean_squared_error', cv = 10)

res_df = DataFrame(res)

res_df

In [ ]:
res_df.mean()

It appears that a small $\alpha$ leads to smaller MSE. Thus I will choose $\alpha = 0.125$. Let's now see how ridge regression with this chosen $\alpha$ performs on the test set.

In [ ]:
ridge3 = Ridge(alpha=0.125)
ridge3.fit(data_train, price_train)

testpredprice = ridge3.predict(data_test)
mean_squared_error(price_test, testpredprice)

This is higher than the corresponding MSE obtained via OLS; ridge regression does not appear to be a superior choice.